In [ ]:
import collections
import heapq
import itertools
import os
import shutil
import string

import numpy as np
import pandas as pd

In [ ]:
class Problem(object):
    
    def __init__(self, problem_name, has_subtests=True):
        self.problem_name = problem_name
        self.test_id = 0
        self.has_subtests = has_subtests
        if os.path.exists(self.problem_name):
            shutil.rmtree(self.problem_name)
        os.makedirs(f'{self.problem_name}/input')
        os.makedirs(f'{self.problem_name}/output')
        
    def create_inputs(self, **kwargs):
        """Should return a list. Each element is a line in the input."""
        raise NotImplementedError
    
    def solve(self, inputs, **kwargs):
        """Should return a list. Each element is a line in the output."""
        raise NotImplementedError
    
    @staticmethod
    def save_to_file(data, file_path=None, **kwargs):
        if isinstance(data, list):
            data = ' '.join(str(x) for x in data)
        print(data, file=file_path, **kwargs)
    
    def create_single_test(self, inputs=None, **kwargs):
        n = 1
        if inputs is None:
            if self.has_subtests:
                if 'N' in kwargs:
                    n = kwargs['N']
                else:
                    n = np.random.randint(kwargs['N_MIN'], kwargs['N_MAX']+1)
                inputs = [n]
                outputs = []
                for _ in range(n):
                    inp = self.create_inputs(**kwargs)
                    outp = self.solve(inp)
                    inputs += inp
                    outputs += outp
            else:
                inputs = self.create_inputs(**kwargs)
                outputs = self.solve(inputs)
        else:
            outputs = self.solve(inputs, **kwargs)
            if self.has_subtests:
                inputs = [1] + inputs

        self.write_test(self.test_id, inputs, outputs)
        self.test_id += 1
        return (self.test_id-1, inputs, outputs)
    
    def create_multiple_tests(self, n, **kwargs):
        for i in range(n):
            yield self.create_single_test(**kwargs)
            
    def write_test(self, test_id, inputs, outputs):
        input_file = f'{self.problem_name}/input/input_{test_id:02d}.txt'
        output_file = f'{self.problem_name}/output/output_{test_id:02d}.txt'

        with open(input_file, 'w') as fp:
            for line in inputs:
                Problem.save_to_file(line, fp)

        with open(output_file, 'w') as fp:
            for line in outputs:
                Problem.save_to_file(line, fp)
    
    def create_from_topcoder(self, filepath, input_parser=None, output_parser=None, max_test=50, **kwargs):
        data = pd.read_csv(filepath, sep='\t', header=None).sample(frac=1)
        for i in range(min(len(data), max_test)):
            inputs = data.iloc[i, 0] if input_parser is None else input_parser(data.iloc[i, 0])
            outputs = data.iloc[i, 1] if output_parser is None else output_parser(data.iloc[i, 1])
            self.write_test(i, inputs, outputs)
            self.test_id += 1
            yield (i, inputs, outputs)
    

def parse_topcoder_data(data):
    data = data.replace('{', '[').replace('}', ']')
    data = eval(data)
    if not isinstance(data, tuple):
        data = [data]
    return data

In [ ]:
class PandemicSimulator(Problem):
    def __init__(self):
        super().__init__('pandemic-simulator', False)

    def create_inputs(self, n_points=10000, r=1000, c=1000, **kwargs):
        r = np.random.randint(r>>1, r+1)
        c = np.random.randint(c>>1, c+1)
        n = np.random.randint(n_points>>1, n_points+1)
        idx = list(range(r*c))
        np.random.shuffle(idx)
        coors = [[np.random.randint(101), x//c, x%c] for x in idx[:n]]
        return [[r, c, n]] + coors
    
    def solve(self, inputs, **kwargs):
        (r, c, n), *points = inputs
        priority_queue = []
        matrix = [[0]*c for _ in range(r)]
        for p, i, j in points:
            heapq.heappush(priority_queue, (-p, i, j))
            matrix[i][j] = -p
            
        # BFS
        while priority_queue:
            p, i, j = heapq.heappop(priority_queue)
            if p == -1:
                continue
            for ii, jj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ni, nj = i+ii, j+jj
                if 0 <= ni < r and 0 <= nj < c and matrix[ni][nj] == 0:
                    heapq.heappush(priority_queue, (p+1, ni, nj))
                    matrix[ni][nj] = p+1
        return [[-p for p in row] for row in matrix]

In [ ]:
problem = PandemicSimulator()

In [ ]:
problem.create_single_test([
    [5, 6, 3],
    [1, 1, 0],
    [2, 2, 5],
    [4, 3, 1],
])

In [ ]:
_ = problem.create_single_test([
    [1, 1, 1],
    [100, 0, 0],
])

In [ ]:
_ = problem.create_single_test([
    [5000, 5000, 0],
])

In [ ]:
problem.create_single_test([
    [10, 10, 1],
    [100, 0, 0],
])

In [ ]:
problem.create_single_test([
    [10, 10, 1],
    [10, 0, 0],
])

In [ ]:
problem.create_single_test([
    [3, 3, 9],
    [1, 0, 0],
    [1, 0, 1],
    [1, 0, 2],
    [1, 1, 0],
    [1, 1, 1],
    [1, 1, 2],
    [1, 2, 0],
    [1, 2, 1],
    [1, 2, 2],
])

In [ ]:
problem.create_single_test([
    [3, 3, 9],
    [1, 0, 0],
    [1, 0, 1],
    [1, 0, 2],
    [1, 1, 0],
    [3, 1, 1],
    [1, 1, 2],
    [1, 2, 0],
    [1, 2, 1],
    [1, 2, 2],
])

In [ ]:
_ = problem.create_single_test([
    [100, 100, 4],
    [100, 0, 0],
    [100, 0, 99],
    [100, 99, 0],
    [100, 99, 99],
])

In [ ]:
for _ in problem.create_multiple_tests(3, n_points=4, r=6, c=6): pass
for _ in problem.create_multiple_tests(5, n_points=5, r=10, c=10): pass
for _ in problem.create_multiple_tests(5, n_points=50, r=10, c=10): pass
for _ in problem.create_multiple_tests(5, n_points=50, r=20, c=20): pass
for _ in problem.create_multiple_tests(5, n_points=500, r=50, c=50): pass
for _ in problem.create_multiple_tests(5, n_points=5000, r=300, c=300): pass
for _ in problem.create_multiple_tests(5, n_points=10000, r=1000, c=1000): pass